<a href="https://colab.research.google.com/github/thor4/crypto/blob/main/dhramm/json_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coingecko Pricing Data CSV Creator

1.   Download historical pricing data using the Coingecko API.
- - https://www.coingecko.com/en/api/documentation
- - ie: https://api.coingecko.com/api/v3/coins/ethereum/market_chart/range?vs_currency=usd&from=1630387200&to=1676784000
2.   Upload the JSON as data.json to the working directory.
3.   The following script will extract price and convert all dates then save as a CSV with date and price columns.

In [1]:
import json
import csv
from datetime import datetime
import pytz

# load the JSON data from a file
with open('data.json') as f:
    data = json.load(f)

# extract the "prices" data from the JSON
prices = data['prices']

# create a timezone object for Eastern Time
eastern_tz = pytz.timezone('US/Eastern')

# convert the UNIX timestamps to date strings and create a list of dictionaries
price_data = []
for item in prices:
    timestamp = item[0] // 1000  # convert to seconds
    date = datetime.utcfromtimestamp(timestamp).replace(tzinfo=pytz.UTC)
    date = date.astimezone(eastern_tz)
    date_string = date.strftime('%Y-%m-%d')
    # price = item[1]
    # price_data.append({'date': date_string, 'price': price})
    # extract only dates between [2/14/2022, 2/18/23], inclusive
    if datetime(2022, 2, 14, 0, 0, tzinfo=eastern_tz) <= date <= datetime(2023, 2, 18, 23, 59, 59, tzinfo=eastern_tz):
        date_string = date.strftime('%Y-%m-%d')
        price = item[1]
        price_data.append({'date': date_string, 'price': price})

# write the prices data to a CSV file
with open('prices.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['date', 'price'])
    writer.writeheader()
    for item in price_data:
        writer.writerow(item)


If needed, interpolate pricing for any missing dates.

In [2]:
import pandas as pd

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('prices.csv')

# Convert the date column to a datetime object
df['date'] = pd.to_datetime(df['date'])

# Set the date column as the DataFrame index
df = df.set_index('date')

# Create a new DataFrame with all dates between the minimum and maximum date
min_date = df.index.min()
max_date = df.index.max()
new_index = pd.date_range(min_date, max_date)
new_df = pd.DataFrame(index=new_index)

# Merge the new DataFrame with the original DataFrame
merged_df = pd.merge(new_df, df, how='left', left_index=True, right_index=True)

# Interpolate missing values
interpolated_df = merged_df.interpolate(method='time')

# Save the interpolated prices to a new CSV file
interpolated_df.to_csv('interpolated_prices.csv')
